# 🔄 Chain of Draft (CoD) 프롬프팅 기법

## 📝 Chain of Draft란 무엇인가?

Chain of Draft(CoD)는 LLM(대규모 언어 모델)의 추론 과정을 **최소한의 단어로 압축**하여 효율성을 극대화하는 프롬프팅 기법입니다. 이 방식은 각 추론 단계를 **5단어 이내**로 제한하여 토큰 사용량을 줄이면서도 추론 품질은 유지합니다.

CoD는 인간의 문제 해결 행동에서 영감을 받았습니다. 우리가 복잡한 문제를 해결할 때 모든 생각을 자세히 적지 않고 핵심 정보만 간략하게 메모하는 것처럼, CoD도 필수적인 정보만 담은 간결한 단계를 생성합니다.

## 🔍 CoT vs CoD: 무엇이 다른가?

### Chain-of-Thought(CoT) 예시:
```
Q: Jason은 20개의 사탕을 가지고 있었습니다. 그는 Denny에게 몇 개의 사탕을 주었습니다. 이제 Jason은 12개의 사탕을 가지고 있습니다. Jason이 Denny에게 준 사탕은 몇 개인가요?

A: 단계별로 생각해보겠습니다:
1. 처음에 Jason은 20개의 사탕을 가지고 있었습니다.
2. Denny에게 몇 개를 준 후, Jason은 이제 12개의 사탕을 가지고 있습니다.
3. Jason이 Denny에게 준 사탕의 수를 찾기 위해, 초기 사탕 수와 남은 사탕 수의 차이를 계산해야 합니다.
4. 간단한 뺄셈 문제를 설정할 수 있습니다: 초기 사탕 수 - 남은 사탕 수 = Denny에게 준 사탕 수
5. 숫자를 대입하면: 20 - 12 = Denny에게 준 사탕 수
6. 뺄셈을 계산하면: 20 - 12 = 8

따라서, Jason은 Denny에게 8개의 사탕을 주었습니다.

[응답]: 8개의 사탕
```

### Chain-of-Draft(CoD) 예시:
```
Q: Jason은 20개의 사탕을 가지고 있었습니다. 그는 Denny에게 몇 개의 사탕을 주었습니다. 이제 Jason은 12개의 사탕을 가지고 있습니다. Jason이 Denny에게 준 사탕은 몇 개인가요?

A: 20 - x = 12; x = 20 - 12 = 8.

[응답]: 8
```

## ✨ CoD의 장점

- **🚀 토큰 사용량 감소**: CoT 대비 68-92%의 토큰 절약
- **⏱️ 지연 시간 단축**: 응답 생성 속도 40-76% 향상
- **💰 비용 효율성**: 대량 애플리케이션에서 상당한 비용 절감
- **🎯 정확도 유지**: 간결함에도 불구하고 CoT와 유사한 정확도 유지

## 📊 성능 벤치마크

| 모델 | 작업 | CoT 정확도 | CoD 정확도 | 토큰 감소율 | 지연 시간 감소 |
|------|------|------------|------------|-------------|---------------|
| GPT-4o | GSM8k | 95.4% | 91.1% | 80% | 76.2% |
| Claude 3.5 | Sports | 93.2% | 97.3% | 92.4% | 72.2% |
| GPT-4o | Coin Flip | 100% | 100% | 68% | 42.9% |



### CoD는 어떤 작업에 가장 적합한가요?
🧩 CoD는 수학적 문제 해결, 기호적 추론, 논리 문제와 같은 구조화된 추론 작업에 이상적입니다.

### CoD의 한계는 무엇인가요?
🚧 CoD는 제로샷 설정과 작은 모델(3B 파라미터 미만)에서 어려움을 겪습니다. 상세한 설명이 필요한 매우 복잡한 문제는 여전히 전통적인 Chain of Thought 접근 방식이 유리할 수 있습니다.

### CoD는 다른 프롬프팅 기법과 결합될 수 있나요?
🔄 네, CoD는 few-shot 학습, self-consistency 샘플링 등과 잘 통합됩니다. 결합된 접근 방식으로 실험하면 성능을 더욱 최적화할 수 있습니다.

## 🚀 결론

Chain of Draft는 LLM의 추론 능력을 유지하면서 효율성을 크게 향상시키는 혁신적인 프롬프팅 기법입니다. 간결함의 힘을 활용하여 토큰 사용량, 지연 시간, 비용을 줄이면서도 정확도는 유지합니다. 실시간 애플리케이션과 대규모 데이터 처리 시나리오에서 특히 유용한 이 기법은 AI 시스템의 실용적인 배포를 위한 중요한 발전을 제공합니다.



## 테스트 코드

In [27]:
import boto3
from botocore.exceptions import ClientError

def invoke_bedrock_model(cod_prompt, user_query, model_id="amazon.nova-pro-v1:0", region_name="us-east-1"):
    """
    Invokes Bedrock model with the given system prompt and user query.
    
    Args:
        cod_prompt (str): System prompt for Chain of Draft
        user_query (str): User's query to process
        model_id (str): Bedrock model ID to use
        region_name (str): AWS region name
        
    Returns:
        str: Model's response text
    """
    # Create a Bedrock Runtime client
    client = boto3.client("bedrock-runtime", region_name=region_name)
    
    try:
        # Send the message to the model with CoD instructions as system prompt
        response = client.converse(
            modelId=model_id,
            messages=[
                {
                    "role": "user",
                    "content": [{"text": user_query}]
                }
            ],
            system=[{"text": cod_prompt}],
            inferenceConfig={
                "maxTokens": 512, 
                "temperature": 0.7,
                "topP": 0.9
            }
        )

        # Extract and return the response text
        response_text = response["output"]["message"]["content"][0]["text"]
        return response_text

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

# Chain of Draft prompt
cod_prompt = "Think step by step, but only keep a minimum draft for each thinking step, with 5 words at most. Return the answer at the end of the response after a separator ####."

# User query
user_query = "If a train travels at 60 miles per hour for 2.5 hours, how far does it go?"

# Call the function and print the result
result = invoke_bedrock_model(cod_prompt, user_query)
print(result)



Distance equals speed times time, simple multiplication.

#### 150 miles traveled by the train.


## Chain of Throught의 한계

In [46]:
# Chain of Draft prompt
cod_prompt = """
다음 질문에 답하기 위해 단계별로 생각해 보세요. 응답이 끝난 후 답변을 반환합니다. 답변 구분 기호 ####
"""

# User query
user_query = """
연욱이는 막대 사탕 20개를 가지고 있었습니다. 연욱이는 효원이에게 막대사탕을 조금 주었습니다. 이제 12개가 남았습니다. 연욱이는  효원이에게 몇 개를 주었나요?
"""

# Call the function and print the result
result = invoke_bedrock_model(cod_prompt, user_query)
print(result)

연욱이가 효원이에게 준 막대 사탕의 개수를 구하기 위해서는 다음과 같은 단계를 따라야 합니다.

1. 연욱이가 원래 가지고 있던 막대 사탕의 개수는 20개입니다.
2. 연욱이가 효원이에게 막대 사탕을 준 후 남은 막대 사탕의 개수는 12개입니다.
3. 연욱이가 효원이에게 준 막대 사탕의 개수는 원래 가지고 있던 개수에서 남은 개수를 뺀 값입니다.

따라서, 연욱이가 효원이에게 준 막대 사탕의 개수를 계산하면 다음과 같습니다:
20개 - 12개 = 8개

연욱이는 효원이에게 8개의 막대 사탕을 주었습니다.

#### 8


# Chain of Draft

In [47]:
# Chain of Draft prompt
cod_prompt = """
단계별로 생각하되 최소한의 초안만 유지하세요. 각 단계마다 최대 5단어로 유지합니다. 답변이 끝난 후 구분하여 답변을 반환합니다. 답변 구분 기호 ####.
"""

# User query
user_query = """
연욱이는 막대 사탕 20개를 가지고 있었습니다. 연욱이는 효원이에게 막대사탕을 조금 주었습니다. 이제 12개가 남았습니다. 연욱이는  효원이에게 몇 개를 주었나요?
"""

# Call the function and print the result
result = invoke_bedrock_model(cod_prompt, user_query)
print(result)

ERROR: Can't invoke 'amazon.nova-pro-v1:0'. Reason: An error occurred (ThrottlingException) when calling the Converse operation (reached max retries: 4): Too many requests, please wait before trying again.
None


# 퀴즈

### 1초 이내에 답변이 정확히 나오려면 어떻게 해야될까요?

In [61]:
import boto3
from botocore.exceptions import ClientError

def invoke_bedrock_model(cod_prompt, user_query, model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name="us-east-1"):
    """
    Invokes Bedrock model with the given system prompt and user query.
    
    Args:
        cod_prompt (str): System prompt for Chain of Draft
        user_query (str): User's query to process
        model_id (str): Bedrock model ID to use
        region_name (str): AWS region name
        
    Returns:
        str: Model's response text
    """
    # Create a Bedrock Runtime client
    client = boto3.client("bedrock-runtime", region_name=region_name)
    
    try:
        # Send the message to the model with CoD instructions as system prompt
        response = client.converse(
            modelId=model_id,
            messages=[
                {
                    "role": "user",
                    "content": [{"text": user_query}]
                }
            ],
            system=[{"text": cod_prompt}],
            inferenceConfig={
                "maxTokens": 512, 
                "temperature": 0.7,
                "topP": 0.9
            }
        )

        # Extract and return the response text
        response_text = response["output"]["message"]["content"][0]["text"]
        return response_text

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

In [64]:
cod_prompt = """
단계별로 생각하되 최소한의 초안만 유지하세요. 각 단계마다 최대 5단어로 유지합니다. 답변이 끝난 후 구분하여 답변을 반환합니다. 답변 구분 기호 ####.
"""

# User query
user_query = """
"한 상점에서 20% 할인 행사를 하고 있습니다. 원래 가격이 50달러인 셔츠를 사려고 합니다. 할인 후 가격은 얼마이며, 얼마를 절약할 수 있나요?"
"""

# Call the function and print the result
result = invoke_bedrock_model(cod_prompt, user_query)
print(result)

1. 할인율: 20%
2. 원가격: 50달러
3. 할인액: 10달러
4. 할인후가격: 40달러
5. 절약액: 10달러

####.

셔츠의 할인 후 가격은 40달러이며, 절약할 수 있는 금액은 10달러입니다.
